# Robust Model Zoo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/metinaktas/Robust-Model-Zoo/blob/main/CIFAR-10/robust-model-zoo-cifar10-tf1.ipynb#scrollTo=ZvP3YCCTCH56)

This notebook is used to measure robustness accuracy of the model trained for CIFAR-10 dataset. We used [AutoAttack](https://github.com/fra31/auto-attack) repository for robustness measurement.


## Install packages

In [ ]:
# Install gdown for google drive download
! conda install -y gdown
# Install AutoAttack
! pip install git+https://github.com/fra31/auto-attack

/bin/bash: conda: command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/fra31/auto-attack to /tmp/pip-req-build-6784w4r4
  Running command git clone -q https://github.com/fra31/auto-attack /tmp/pip-req-build-6784w4r4
  Created wheel for autoattack: filename=autoattack-0.1-py3-none-any.whl size=34538 sha256=f26b4a321cdcc2b3e46cbe238c02dde849489cd4572bbed0349e513c1584da82
  Stored in directory: /tmp/pip-ephem-wheel-cache-dsppxaah/wheels/a2/71/8b/ec4bb51ceac509961546bf5430b7cf433f93bd375ffa75ea45
Successfully built autoattack


In [ ]:
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
import numpy as np
import sys
import os
from pathlib import Path
import tarfile
import shutil
import gdown

Instructions for updating:
non-resource variables are not supported in the long term


# Download required files for evaluation

Create folder structures

In [ ]:
base_folder = "CIFAR-10"
dataset_folder = base_folder+"/"+"Dataset"
model_folder = base_folder+"/"+"Models/tf1"

In [ ]:
if Path(base_folder).exists() == False:
    os.makedirs(base_folder)
if Path(dataset_folder).exists() == False:
    os.makedirs(dataset_folder)
if Path(model_folder).exists() == False:
    os.makedirs(model_folder)
if Path("./auto-attack").exists() == False:
    os.makedirs("./auto-attack") 

In [ ]:
# model_1 in https://github.com/metinaktas/Robust-Model-Zoo/blob/main/CIFAR-10/Models/tf1/model_1/model.py
#model_info = {'model':"https://drive.google.com/uc?export=download&id=1CXqynYUS3E_BOfQ6LfBT4bxN2kbLBAAY",'checkpoint':"https://drive.google.com/uc?export=download&id=16PLzHqErmNdPHexlPfZ8ccxeIaqWOMTt"}

# model_2 in https://github.com/metinaktas/Robust-Model-Zoo/blob/main/CIFAR-10/Models/tf1/model_2/model.py
model_info = {'model':"https://drive.google.com/uc?export=download&id=1cXSbTcJ5_Bm3yfYlTrhfToRZEjQIkJ8S",'checkpoint':"https://drive.google.com/uc?export=download&id=1s76yDCBK86KN_cxjJM87i4qn897arAY_"}

Download CIFAR-10 dataset

In [ ]:
url = "https://drive.google.com/uc?export=download&id=1lt7yhOVKEVydOupZ1taA3iNXC5Ety79i"
output = dataset_folder+'/dataset_cifar10.zip'
gdown.download(url, output, quiet=False)
shutil.unpack_archive(output, dataset_folder)
os.remove(output)

Downloading...
From: https://drive.google.com/uc?export=download&id=1lt7yhOVKEVydOupZ1taA3iNXC5Ety79i
To: /content/CIFAR-10/Dataset/dataset_cifar10.zip
100%|██████████| 170M/170M [00:02<00:00, 77.6MB/s]


Download trained model

In [ ]:
url = model_info['checkpoint']
output = model_folder+'/checkpoint.zip'
gdown.download(url, output, quiet=False)
shutil.unpack_archive(output, model_folder)
os.remove(output)

Downloading...
From: https://drive.google.com/uc?export=download&id=16PLzHqErmNdPHexlPfZ8ccxeIaqWOMTt
To: /content/CIFAR-10/Models/tf1/checkpoint.zip
100%|██████████| 163M/163M [00:02<00:00, 65.9MB/s]


Download code files

In [ ]:
# Download file for cifar-10 input reading
url = "https://drive.google.com/uc?export=download&id=1eOWDdFbPQ8sQJOg7ueuF-bXaL0ev-SK2"
output = base_folder+'/cifar10_input.py'
gdown.download(url, output, quiet=False)    

# Download file for TF1 model
url = model_info['model']
output = base_folder+'/model_submit.py'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1eOWDdFbPQ8sQJOg7ueuF-bXaL0ev-SK2
To: /content/CIFAR-10/cifar10_input.py
100%|██████████| 6.83k/6.83k [00:00<00:00, 1.18MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1CXqynYUS3E_BOfQ6LfBT4bxN2kbLBAAY
To: /content/CIFAR-10/model_submit.py
100%|██████████| 12.0k/12.0k [00:00<00:00, 21.9MB/s]


'CIFAR-10/model_submit.py'

Download the AutoAttack clone (Commits on Jun 22, 2022)

In [ ]:
url = "https://drive.google.com/uc?export=download&id=1S7ivr2avGlK28Aw9fAbavDwaOejX2yt9"
output = "./auto-attack"+'/auto-attack-TF1.zip'
gdown.download(url, output, quiet=False)
shutil.unpack_archive(output, "./auto-attack")
os.remove(output) 

Downloading...
From: https://drive.google.com/uc?export=download&id=1S7ivr2avGlK28Aw9fAbavDwaOejX2yt9
To: /content/auto-attack/auto-attack-TF1.zip
100%|██████████| 42.4M/42.4M [00:01<00:00, 26.4MB/s]


# ROBUSTNESS TEST

Modify autoattack repository after installation to run TF1 codes on TF2 backend.

Modifications on "utils_tf.py" are,
* Line 17: self.grads[cl] = tf.gradients(self.logits[:, cl], self.x_input)[0] ==> self.grads[cl] = tf.compat.v1.gradients(self.logits[:, cl], self.x_input)[0]
* Line 22: self.grad_xent = tf.gradients(self.xent, self.x_input)[0] ==> self.grad_xent = tf.compat.v1.gradients(self.xent, self.x_input)[0]
* Line 26: self.grad_dlr = tf.gradients(self.dlr, self.x_input)[0] ==> self.grad_dlr = tf.compat.v1.gradients(self.dlr, self.x_input)[0]
* Line 29: self.y_target = tf.placeholder(tf.int64, shape=[None]) ==> self.y_target = tf.compat.v1.placeholder(tf.int64, shape=[None])
* Line 32: self.grad_target = tf.gradients(self.dlr_target, self.x_input)[0] ==> self.grad_target = tf.compat.v1.gradients(self.dlr_target, self.x_input)[0]
* Line 33: self.la = tf.placeholder(tf.int64, shape=[None]) ==> self.la = tf.compat.v1.placeholder(tf.int64, shape=[None])
* Line 34: self.la_target = tf.placeholder(tf.int64, shape=[None]) ==> self.la_target = tf.compat.v1.placeholder(tf.int64, shape=[None])
* Line 40: self.grad_diff_logits = tf.gradients(self.diff_logits, self.x_input)[0] ==> self.grad_diff_logits = tf.compat.v1.gradients(self.diff_logits, self.x_input)[0]
* Line 91: tf.contrib.framework.sort(x, axis=1) ==> x_sort = tf.sort(x, axis=1)
* Line 99: tf.contrib.framework.sort(x, axis=1) ==> x_sort = tf.sort(x, axis=1)

Modifications on "autoattack.py" are,
* Line 91 and 92 are comment out since "check_dynamic" function in checks.py has not implemented to support TF1 codes. This is handled with 'the check for dynamic defenses is not currently supported' message, but model(x) function call in line 101 on checks.py is not handled for TF1 and gives error.

The proposed model contains "quantization" blocks for realizing gradient masking. To evaluate the robustness of the proposed model fairly, we should take into account the existence of "quantization" blocks and help backpropagation to find gradients. The [AutoAttack flag_docs](https://github.com/fra31/auto-attack/blob/master/flags_doc.md) suggest to use [BPDA](http://proceedings.mlr.press/v80/athalye18a.html) method, which approximates such functions with differentiable counterparts. Since BPDA is not implemented in AutoAttack, we modify the attack codes to handle different models in feedforward and backpropagation phases. With these modifications, we can use original proposed model (with quantization blocks) in evalution and model without quantization blocks in computing gradients during adversarial example generation.

In [ ]:
!cp -r /content/auto-attack/auto-attack-TF1_Heterogeneous/autoattack/. /usr/local/lib/python3.7/dist-packages/autoattack/

In [ ]:
sys.path.append("./CIFAR-10")
sys.path.append("./auto-attack/auto-attack-TF1/autoattack")
from model_submit import Model
import cifar10_input
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms
from autoattack import AutoAttack, utils_tf

In [ ]:
# Define adversarial attack
num_examples = 1000
batch_size = 100
epsilon = 8/255.0

# Get model
x_input_place_holder = tf.compat.v1.placeholder(tf.float32,shape=[None, 32, 32, 3],name='x_input')
y_input_place_holder = tf.compat.v1.placeholder(tf.int64, shape=None)
model_quantize = Model(mode='eval',x_input=x_input_place_holder,y_input=y_input_place_holder,quantize_active=True)
model_no_quantize = Model(mode='eval',x_input=x_input_place_holder,y_input=y_input_place_holder,quantize_active=False)
saver = tf.compat.v1.train.Saver()

# Dataset read functions
cifar = cifar10_input.CIFAR10Data(dataset_folder)

# load data
testX = cifar.eval_data.xs[0:num_examples,:,:,:]/255.0
testY = cifar.eval_data.ys[0:num_examples]

# convert to pytorch format
torch_testX = torch.from_numpy( np.transpose(testX, (0, 3, 1, 2)) ).float().cuda()
torch_testY = torch.from_numpy( testY ).cuda()

# load model from saved weights
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))
# Restore the model
model_checkpoint = tf.train.latest_checkpoint(model_folder)
saver.restore(sess, model_checkpoint)

model_adapted = utils_tf.ModelAdapter(model_quantize.pre_softmax, model_quantize.x_input, model_quantize.y_input, sess)
model_back_adapted = utils_tf.ModelAdapter(model_no_quantize.pre_softmax, model_no_quantize.x_input, model_no_quantize.y_input, sess)

# run plus attack
print("******** RUN PLUS ATTACK *******")
adversary = AutoAttack(model_back_adapted,model_adapted, norm='Linf', eps=epsilon, version='plus', is_tf_model=True,verbose=True)
x_adv = adversary.run_standard_evaluation(torch_testX, torch_testY, bs=batch_size)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from CIFAR-10/Models/tf1/checkpoint-0
******** RUN PLUS ATTACK *******
setting parameters for plus version
using plus version including apgd-ce, apgd-dlr, fab, square, apgd-t, fab-t
initial accuracy: 77.70%
apgd-ce - 1/8 - 15 out of 100 successfully perturbed
apgd-ce - 2/8 - 10 out of 100 successfully perturbed
apgd-ce - 3/8 - 16 out of 100 successfully perturbed
apgd-ce - 4/8 - 15 out of 100 successfully perturbed
apgd-ce - 5/8 - 6 out of 100 successfully perturbed
apgd-ce - 6/8 - 15 out of 100 successfully perturbed
apgd-ce - 7/8 - 11 out of 100 successfully perturbed
apgd-ce - 8/8 - 8 out of 77 successfully perturbed
robust accuracy after APGD-CE: 68.10% (total time 1637.1 s)
apgd-dlr - 1/7 - 11 out of 100 successfully perturbed
apgd-dlr - 2/7 - 6 out of 100 successfully perturbed
apgd-dlr - 3/7 - 9 out of 100 successfully perturb